In [ ]:
import os
expID = "exp1"
# Get the current working directory
current_path = os.getcwd()
# Get the parent directory path
result_dir = os.path.join(current_path, "exps/"+expID)
os.makedirs(result_dir, exist_ok=True)

In [ ]:
## 1️⃣ Drone Signal Generation
## copy the config file from drone_generator/exp_config.yaml to exps/exp1/, then edit it to adjust parameters
!python drone_generator/drone_generator_main.py --num 20 --exp "$expID" --drone_data DroneAudioData

In [ ]:
# for drone test dataset
!python drone_generator/drone_generator_main.py --num 10 --exp "$expID" --drone_data DroneAudioDataTest

In [ ]:
## 2️⃣ Microphone Array Configuration
# copy the mic_configxxx.yaml (or individual configuration) from mic array to exp folder
!python mic_array/mic_generator_main.py --yaml_file reusev301.yaml --exp "$expID" 

In [ ]:
## 3️⃣ Data Generation
!python data_generator/data_generator_main.py --exp "$expID" --env_path 'env/Reusev301_train' --output 'MicArrayData' --drone_data DroneAudioData

In [ ]:
# test dataset
!python data_generator/data_generator_main.py --exp "$expID" --env_path 'env/Reusev301_test' --output 'MicArrayDataTest' --drone_data DroneAudioDataTest

In [ ]:
## 4️⃣ Neural Network DOA Estimation
!python neural_doa/neural_main.py --exp exp1 \
  --input_feature GCC_PHAT \
  --nn_model SELD_ACCDOA \
  --data_file MicArrayData \
  --data_file_test MicArrayDataTest \
  --is_training
# !CUDA_VISIBLE_DEVICES=1 python neural_doa/neural_main.py --exp "$expID" --input_feature 'GCC_PHAT' --nn_model 'SELD_ACCDOA' --data_file MicArrayData --data_file_test MicArrayDataTest --is_training

In [ ]:
## Evaluate on real-world drone recordings
!python neural_doa/neural_main.py \
  --exp exp1 \
  --input_feature GCC_PHAT \
  --nn_model SELD_ACCDOA \
  --data_file_test MicArrayDataReal30s \
  --only_azi

In [ ]:
## 5️⃣ Traditional DOA Estimation (SRP-PHAT)
!python doa_estimator/doa_runner_main.py --exp "$expID" --algorithm srp_phat --dataset MicArrayDataTest #--mask #--beta 

In [ ]:
## 6️⃣ Evaluation for Traditional DOA Estimation Methods
!python evaluation/eval_runner_main.py --exp "$expID" --eval_alg srp_phat --dataset MicArrayDataTest #--mask --beta 